# Implementing logistic regression to build a Spam Filter

The goal of this workshop is to implement your own logistic regression classifier and build a Spam filter. You will:

 * Extract features from Email Spam database.
 * Convert an DataFrame into a NumPy array.
 * Implement the link function for logistic regression.
 * Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
 * Implement gradient ascent.
 * Given a set of coefficients, predict spam or non-spam.
 * Compute classification accuracy for the logistic regression model.
 
Let's get started!

In [26]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Load review dataset
For this application, we will use a email spam database from UCI data repos. The subset was chosen to contain similar numbers of spam and non spam messages

In [36]:
email = pd.read_csv('email_fresh_1953', sep=',')
# email = pd.read_csv('email_collection', sep='\t', names=["type", "message"])

# email_ham_messages = email[email['type']=='ham'].head(1206)
# email_spam_messages = email[email['type']=='spam']

# email_messages_fresh = email_spam_messages.append(email_ham_messages,ignore_index = True)
# email_messages_fresh.to_csv("email_fresh_1953",sep=',',index=False)

#email['label'] = email['type'].apply(lambda type : +1 if type == 'ham' else -1)
#email_spam_messages['message'].to_csv("email_spam_messages_1953",sep=',',index=False)
#email_ham_messages = email[email['label']==+1]
#email_ham_messages['message'].to_csv("email_ham_messages_1953",sep=',',index=False)


#email = email.reindex(np.random.permutation(email.index))
email

,type,message
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
5,spam,URGENT! You have won a 1 week FREE membership ...
6,spam,"XXXMobileMovieClub: To use your credit, click ..."
7,spam,England v Macedonia - dont miss the goals/team...
8,spam,Thanks for your subscription to Ringtone UK yo...
9,spam,07732584351 - Rodger Burns - MSG = We tried to...


One column of this dataset is 'label', corresponding to the class label with +1 indicating a non-spam message and -1 indicating a spam message.

In [37]:
email['label'] = email['type'].apply(lambda type : +1 if type == 'ham' else -1)
email

,type,message,label
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,-1
1,spam,FreeMsg Hey there darling it's been 3 week's n...,-1
2,spam,WINNER!! As a valued network customer you have...,-1
3,spam,Had your mobile 11 months or more? U R entitle...,-1
4,spam,"SIX chances to win CASH! From 100 to 20,000 po...",-1
5,spam,URGENT! You have won a 1 week FREE membership ...,-1
6,spam,"XXXMobileMovieClub: To use your credit, click ...",-1
7,spam,England v Macedonia - dont miss the goals/team...,-1
8,spam,Thanks for your subscription to Ringtone UK yo...,-1
9,spam,07732584351 - Rodger Burns - MSG = We tried to...,-1


Let us quickly explore more of this dataset.  The 'name' column indicates the name of the product.  Here we list the first 10 products in the dataset.  We then count the number of positive and negative reviews.

In [38]:
print '# of non-spam messages =', len(email[email['label']==1])
print '# of spam messages =', len(email[email['label']==-1])

# of non-spam messages = 1206
# of spam messages = 747


## Apply text cleaning on the message data

In this section, we will perform some simple feature cleaning using dataframes. We compiled a list of most frequent words into a JSON file.

Now, we will load these words from this JSON file:


In [39]:
import json
with open('important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]
print important_words

['all', 'code', 'sch', '1500', 'TEXT', 'AGE', 'month', 'More', 'Does', 'sleep', 'oso', 'go', 'liao', 'O2', 'Today', 'content', 'Someone', 'tv', 'send', 'to', 'finally', 'does', 'toClaim', 'Not', 'smile', 'Latest', 'worth', 'sent', 'And', 'yr', 'chennai', 'Prize', 'very', 'Sky', 'Last', 'anyway', '150p', 'every', '2000', 'Aight', 'word', 'ticket', '12hrs', 'Where', 'minute', 'cool', 'Enjoy', 'school', 'prize', 'Just', 'did', 'gettin', 'eve', 'No1', 'leave', 'ONLY', 'Text', 'Go', 'Ufind', 'havent', 'Pound', 'TsCs', 'dis', 'nokia', 'TO', 'says', 'specially', 'videochat', 'tired', 'phones', 'Terms', 'direct', 'Should', 'second', 'cost', 'video', 'voucher', 'picked', 'pass', 'download', 'run', 'rentl', '3G', 'Expires', 'blue', '9AE', 'Calls', 'rate', 'giving', 'Videophones', 'selected', 'access', 'waiting', 'CAMERA', 'abt', 'SIM', 'goes', 'reply', 'net', 'told', 'Me', 'full', 'TONE', 'Gift', 'leh', 'Yup', 'here', 'Congratulations', 'Ibiza', 'met', 'FLAG', 'let', 'pub', 'making', 'Alright', 

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Compute word counts (only for **important_words**)

We start with *Step 1* which can be done as follows:

In [40]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

email['message_clean'] = email['message'].apply(remove_punctuation)

Now we proceed with Step 2. For each word in important_words, we compute a count for the number of times the word occurs in each message. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in important_words which keeps a count of the number of times the respective word occurs in the review text.
(also known as one-hot encoding.)


Note: There are several ways of doing this. In this assignment, we use the built-in count function for Python lists. Each review string is first split into individual words and the number of occurances of a given word is counted.


In [41]:
# this might take a while
for word in important_words:
    email[word] = email['message_clean'].apply(lambda s : s.split().count(word))

The dataFrame **email** now contains one column for each of the **important_words**. As an example, the column **"FREE"** contains a count of the number of times the word **"FREE"** occurs in each of the messages.

In [42]:
len(email[email['FREE'] >= 1])

93

## Convert DataFrame to NumPy array

As you have seen previously, NumPy is a powerful library for doing matrix manipulation. Let us convert our data to matrices and then implement our algorithms with matrices.

First, make sure you can perform the following import.

In [43]:
import numpy as np

We now provide you with a function that extracts columns from an DataFrame and converts them into a NumPy array. Two arrays are returned: one representing features and another representing class labels. Note that the feature matrix includes an additional column 'intercept' to take account of the intercept term.

In [44]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.as_matrix()
    label_sarray = data_sframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

Let us convert the data into NumPy arrays.

In [45]:
# Warning: This may take a few minutes...
feature_matrix, label = get_numpy_data(email, important_words, 'label') 

In [46]:
feature_matrix.shape

(1953, 1135)

Now, let us see what the **label** column looks like:

In [47]:
label

array([-1, -1, -1, ...,  1,  1,  1])

## Estimating conditional probability with link function
Recall from lecture that the link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review  $\mathbf{x}_i$. Complete the following function that implements the link function:

In [48]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    hx = np.dot(feature_matrix,coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1.0/(1 + np.exp(-hx))
    
    # return predictions
    return predictions

**Aside**. How the link function works with matrix algebra

Since the word counts are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

## Compute derivative of log likelihood with respect to a single coefficient

Recall from lecture:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

We will now write a function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

Complete the following code block:

In [65]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors,feature)
    # Return the derivative
    return derivative

[False False False ...,  True  True  True]


In the main lecture, our focus was on the likelihood.  In the advanced optional video, however, we introduced a transformation of this likelihood---called the log likelihood---that simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

We provide a function to compute the log likelihood for the entire dataset. 

In [66]:
def compute_log_likelihood(feature_matrix, label, coefficients):
    indicator = (label==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]

    lp = np.sum((indicator-1)*scores - logexp)
    return lp

## Taking gradient steps
Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum. 

Complete the following function to solve the logistic regression model using gradient ascent:

In [67]:
from math import sqrt

def logistic_regression(feature_matrix, label, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):

        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix,initial_coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (label==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop over each coefficient
            
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])
            
            # add the step size times the derivative to the current coefficient
            ## YOUR CODE HERE
            coefficients[j] = coefficients[j] + step_size*derivative
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, label, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)
    return coefficients

Now, let us run the logistic regression solver.

In [68]:
coefficients = logistic_regression(feature_matrix, label, initial_coefficients=np.zeros(feature_matrix.shape[1]),
                                   step_size=1e-5, max_iter=951)

[False False False ..., False False False]
iteration   0: log likelihood of observed labels = -1352.39170405
[False False False ..., False False False]
iteration   1: log likelihood of observed labels = -1351.06902028
[False False False ..., False False False]
iteration   2: log likelihood of observed labels = -1349.74839233
[False False False ..., False False False]
iteration   3: log likelihood of observed labels = -1348.42982018
[False False False ..., False False False]
iteration   4: log likelihood of observed labels = -1347.11330380
[False False False ..., False False False]
iteration   5: log likelihood of observed labels = -1345.79884316
[False False False ..., False False False]
iteration   6: log likelihood of observed labels = -1344.48643822
[False False False ..., False False False]
iteration   7: log likelihood of observed labels = -1343.17608894
[False False False ..., False False False]
iteration   8: log likelihood of observed labels = -1341.86779527
[False False False 

KeyboardInterrupt: 

## Predicting the labels

Recall from lecture that class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Now, we will write some code to compute class predictions. We will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [57]:
# Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(feature_matrix, coefficients)
scores
predicted_label = []
for i in range(len(scores)):
    if scores[i] > 0:
        predicted_label.append(1)
    else:
        predicted_label.append(-1)
predicted_label = np.array(predicted_label)
predicted_label
predicted_positive = (predicted_label==+1).sum()

In [58]:
predicted_positive

1624

## Measuring accuracy

We will now measure the classification accuracy of the model. Recall from the lecture that the classification accuracy can be computed as follows:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

Complete the following code block to compute the accuracy of the model.

In [59]:
correct = predicted_label*label
correct = (correct==+1).sum()
num_mistakes = len(predicted_label) - correct 

accuracy = 1.0 * correct / len(label)
print "-----------------------------------------------------"
print '# Reviews   correctly classified =', len(email) - num_mistakes
print '# Reviews incorrectly classified =', num_mistakes
print '# Reviews total                  =', len(email)
print "-----------------------------------------------------"
print 'Accuracy = %.2f' % accuracy

-----------------------------------------------------
# Reviews   correctly classified = 1509
# Reviews incorrectly classified = 444
# Reviews total                  = 1953
-----------------------------------------------------
Accuracy = 0.77


In [60]:
scores = np.dot(feature_matrix, coefficients)
scores

array([-2.306175,  0.60864 , -0.84639 , ...,  1.3314  ,  2.26338 ,
        2.534415])

In [61]:
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

In [62]:
word_coefficient_tuples

[('Im', 0.28054499999999499),
 ('ltgt', 0.26628000000000202),
 ('so', 0.24726000000000431),
 ('got', 0.24726000000000431),
 ('but', 0.24250500000000336),
 ('like', 0.2329949999999969),
 ('come', 0.22348500000000507),
 ('do', 0.21397500000000236),
 ('if', 0.21397500000000236),
 ('Ill', 0.21397500000000236),
 ('go', 0.20446499999999937),
 ('day', 0.19495500000000227),
 ('up', 0.19495500000000227),
 ('need', 0.18068999999999769),
 ('But', 0.17593499999999868),
 ('think', 0.17118000000000319),
 ('ok', 0.17118000000000319),
 ('dont', 0.16642500000000049),
 ('Ok', 0.16167000000000267),
 ('lor', 0.16167000000000267),
 ('time', 0.15691499999999872),
 ('going', 0.15215999999999649),
 ('about', 0.15215999999999649),
 ('there', 0.15215999999999649),
 ('know', 0.14740499999999968),
 ('say', 0.1426499999999998),
 ('So', 0.13789500000000299),
 ('later', 0.13789500000000299),
 ('Sorry', 0.13789500000000299),
 ('much', 0.13789500000000299),
 ('im', 0.13789500000000299),
 ('still', 0.13314000000000101)